# Composable LLM Architecture

This notebook demonstrates hexDAG's **composable LLM architecture** where prompts, LLM calls, and parsing are separate, reusable components.

## Architecture

**Old (Monolithic)**:
```
LLMNode: [Prompt + API Call + Parsing]
```

**New (Composable)**:
```
PromptNode → RawLLMNode → ParserNode
```

## Key Benefits

1. **Separation of Concerns** - Each node does ONE thing
2. **Composable** - Mix and match components
3. **YAML-First** - Declarative pipeline definitions
4. **Type-Safe** - Pydantic validation everywhere
5. **LLM Macro** - Automatic composition for convenience

In [ ]:
# Setup
# Ensure registry is bootstrapped with mock adapters
from hexdag.core.bootstrap import ensure_bootstrapped
from hexdag.core.orchestration.orchestrator import Orchestrator
from hexdag.core.pipeline_builder import YamlPipelineBuilder
from hexdag.core.pipeline_builder.component_instantiator import ComponentInstantiator

ensure_bootstrapped()

# Helper to instantiate ports from config
instantiator = ComponentInstantiator()

## Example 1: Complete YAML Pipeline

Using **PromptNode** + **RawLLMNode** + **ParserNode** with **ports in YAML**.

Everything is declared in YAML - no Python configuration needed!

In [ ]:
# Complete YAML pipeline with ports
complete_pipeline = """
apiVersion: v1
kind: Pipeline
metadata:
  name: qa-pipeline
  description: Question answering with complete YAML configuration

spec:
  # Port configuration - MockLLM for testing  
  ports:
    llm:
      namespace: plugin
      name: mock

  nodes:
    # Step 1: Build prompt
    - kind: prompt_node
      metadata:
        name: build_prompt
      spec:
        template: |
          You are an expert in {{domain}}.
          
          Question: {{question}}
          
          Provide a clear answer in JSON format.
        output_format: messages
        dependencies: []

    # Step 2: Call LLM
    - kind: raw_llm_node
      metadata:
        name: call_llm
      spec:
        dependencies: [build_prompt]

    # Step 3: Parse output
    - kind: parser_node
      metadata:
        name: parse_response
      spec:
        output_schema:
          result: str
        strategy: json
        dependencies: [call_llm]
"""

# Build pipeline
builder = YamlPipelineBuilder()
graph, config = builder.build_from_yaml_string(complete_pipeline)

print(f"✅ Pipeline built with {len(graph.nodes)} nodes")
print(f"📋 Nodes: {list(graph.nodes.keys())}")
print(f"🔌 Ports from YAML: {list(config.ports.keys())}")

In [ ]:
# Execute - instantiate ports from YAML config!
ports = instantiator.instantiate_ports(config.ports)
orchestrator = Orchestrator(ports=ports)

result = await orchestrator.run(
    graph, {"domain": "artificial intelligence", "question": "What is machine learning?"}
)

print("\n📊 Results:")
print(f"Result: {result['parse_response'].result}")

## Example 2: LLM Macro - Automatic Composition

The **LLM Macro** (`core:llm_workflow`) automatically composes PromptNode + RawLLMNode + ParserNode.

Same functionality, more concise YAML!

In [ ]:
# LLM Macro - automatic composition
macro_pipeline = """
apiVersion: v1
kind: Pipeline
metadata:
  name: summarizer-macro

spec:
  ports:
    llm:
      namespace: plugin
      name: mock
  
  nodes:
    - kind: macro_invocation
      metadata:
        name: summarize
      spec:
        macro: core:llm_workflow
        config:
          template: |
            Summarize this text concisely.
            
            Text: {{text}}
            
            Return JSON with 'summary' field.
          output_schema:
            summary: str
          parse_strategy: json
"""

graph2, config2 = builder.build_from_yaml_string(macro_pipeline)
print(f"✅ LLM Macro expanded to {len(graph2.nodes)} nodes")
print(f"📋 Nodes: {list(graph2.nodes.keys())}")
print("🔌 Macro automatically created: PromptNode → RawLLMNode → ParserNode")

In [ ]:
# Execute macro pipeline
ports2 = instantiator.instantiate_ports(config2.ports)
orchestrator2 = Orchestrator(ports=ports2)

result2 = await orchestrator2.run(
    graph2, {"text": "Artificial intelligence is revolutionizing how we work and live."}
)

print("\n📊 Macro Results:")
# Macro creates nodes with _prompt, _llm, _parser suffixes
print(f"Summary: {result2['summarize_parser']}")

## Production YAML Example

In production, your complete YAML would include **ports** configuration:

```yaml
apiVersion: v1
kind: Pipeline
metadata:
  name: production-analyzer

spec:
  # Port configuration (environment-specific)
  ports:
    llm:
      namespace: core
      name: openai
      params:
        api_key: secret:OPENAI_API_KEY
        model: gpt-4
  
  # Execution policies
  policies:
    - kind: retry
      params:
        max_retries: 3
        backoff_factor: 2.0
  
  # Pipeline nodes
  nodes:
    - kind: macro_invocation
      metadata:
        name: analyze
      spec:
        macro: core:llm_workflow
        config:
          template: "Analyze: {{input.text}}"
          output_schema:
            summary: str
          temperature: 0.7
```

Then execute with:
```python
from hexdag.core.pipeline_builder.component_instantiator import ComponentInstantiator

graph, config = builder.build_from_yaml_file("pipeline.yaml")

# Instantiate ports from YAML config
instantiator = ComponentInstantiator()
ports = instantiator.instantiate_ports(config.ports)

# Create orchestrator with instantiated ports
orchestrator = Orchestrator(ports=ports)
result = await orchestrator.run(graph, {"text": "..."})
```

## Summary

The composable LLM architecture with **100% YAML configuration**:

1. ✅ **Separation of Concerns** - Prompt ≠ LLM ≠ Parser
2. ✅ **100% YAML** - Ports, nodes, everything declarative
3. ✅ **Environment-Specific** - Swap `plugin:mock` → `core:openai` in one line
4. ✅ **Type Safety** - Pydantic validation
5. ✅ **Composable** - Mix and match components for different tasks

**YAML-First Benefits**:
- **No Python code** between environments - just swap YAML files
- **dev.yaml** → `llm: plugin:mock`
- **prod.yaml** → `llm: core:openai(...)`
- **CI/CD ready** - Deploy pipelines like infrastructure

**Key Pattern**:
```python
# 1. Build from YAML
graph, config = builder.build_from_yaml_string(yaml_str)

# 2. Instantiate ports (adapters)
ports = instantiator.instantiate_ports(config.ports)

# 3. Run with orchestrator
orchestrator = Orchestrator(ports=ports)
result = await orchestrator.run(graph, inputs)
```

In [ ]:
# DEV spec - fast iteration with mock
dev_spec = """
apiVersion: v1
kind: Pipeline
metadata:
  name: sentiment-analyzer
  namespace: dev

spec:
  ports:
    llm:
      namespace: plugin
      name: mock
  
  nodes:
    - kind: prompt_node
      metadata:
        name: build_prompt
      spec:
        template: "Classify sentiment: {{text}}"
        output_format: messages
        dependencies: []
    
    - kind: raw_llm_node
      metadata:
        name: call_llm
      spec:
        dependencies: [build_prompt]
    
    - kind: parser_node
      metadata:
        name: parse
      spec:
        output_schema:
          result: str
        strategy: json
        dependencies: [call_llm]
"""

# PROD spec - production LLM
prod_spec = """
apiVersion: v1
kind: Pipeline
metadata:
  name: sentiment-analyzer
  namespace: prod

spec:
  ports:
    llm:
      namespace: core
      name: openai
      params:
        api_key: secret:OPENAI_API_KEY
        model: gpt-4
  
  nodes:
    - kind: prompt_node
      metadata:
        name: build_prompt
      spec:
        template: "Classify sentiment: {{text}}\n\nReturn JSON with 'sentiment' and 'confidence' fields."
        output_format: messages
        dependencies: []
    
    - kind: raw_llm_node
      metadata:
        name: call_llm
      spec:
        dependencies: [build_prompt]
    
    - kind: parser_node
      metadata:
        name: parse
      spec:
        output_schema:
          sentiment: str
          confidence: float
        strategy: json
        dependencies: [call_llm]
"""

# Choose environment (in real app: load from file based on ENV var)
import os

env = os.getenv("ENV", "dev")
pipeline_yaml = dev_spec if env == "dev" else prod_spec

graph3, config3 = builder.build_from_yaml_string(pipeline_yaml)

# Instantiate ports from YAML config
ports3 = instantiator.instantiate_ports(config3.ports)

print(f"\n🌍 Environment: {env.upper()}")
print(f"📋 Pipeline: {config3.metadata.get('name')}")
print(f"🔌 Port adapter: {ports3['llm'].__class__.__name__}")
print("\n💡 Same code, different config - that's YAML-first power!")

## Summary

The composable LLM architecture with **100% YAML configuration**:

1. ✅ **Separation of Concerns** - Prompt ≠ LLM ≠ Parser
2. ✅ **100% YAML** - Ports, policies, nodes all declarative
3. ✅ **Environment-Specific** - Swap `core:mock` → `core:openai` in one line
4. ✅ **Type Safety** - Pydantic validation
5. ✅ **Infrastructure as Code** - Git-friendly, reviewable, testable

**YAML-First Benefits**:
- **No Python code** between environments - just swap YAML files
- **dev.yaml** → `llm: core:mock(...)`
- **prod.yaml** → `llm: core:openai(api_key=secret:OPENAI_API_KEY, model=gpt-4)`
- **CI/CD ready** - Deploy pipelines like infrastructure